In [2]:
!pip install onnxruntime

     |████████████████████████████████| 3.1MB 836kB/s eta 0:00:01


In [21]:
!pip install onnx

     |████████████████████████████████| 7.1MB 417kB/s eta 0:00:01


In [3]:
import onnxruntime as ort
import numpy as np
from onnxruntime.datasets import get_example

In [4]:
example1 = get_example("mul_1.pb")

In [5]:
s = ort.InferenceSession(example1)

In [6]:
input_name = s.get_inputs()[0].name

In [7]:
x = np.array([[1.0, 2.0],[3.0, 4.0],[5.0, 6.0]], dtype=np.float32)

In [8]:
r = s.run(None, {input_name : x})

In [9]:
r

[array([[ 1.,  4.],
        [ 9., 16.],
        [25., 36.]], dtype=float32)]

In [10]:
so = ort.SessionOptions()

In [11]:
so.enable_profiling = True

In [12]:
s1 = ort.InferenceSession(example1, so)

In [13]:
input_name1 = s1.get_inputs()[0].name

In [14]:
r1 = s1.run(None, {input_name : x})

In [15]:
p = s1.end_profiling()

In [16]:
print(p)

onnxruntime_profile__2019-06-28_04-34-11.json


In [17]:
import json

In [18]:
with open(p, "r") as f:
    sess_time = json.load(f)

In [19]:
import pprint

In [20]:
pprint.pprint(sess_time)

[{'args': {},
  'cat': 'Session',
  'dur': 99,
  'name': 'model_loading_uri',
  'ph': 'X',
  'pid': 18,
  'tid': 18,
  'ts': 13},
 {'args': {},
  'cat': 'Session',
  'dur': 93,
  'name': 'session_initialization',
  'ph': 'X',
  'pid': 18,
  'tid': 18,
  'ts': 117},
 {'args': {'op_name': 'Mul'},
  'cat': 'Node',
  'dur': 6,
  'name': 'mul_1_fence_before',
  'ph': 'X',
  'pid': 18,
  'tid': 18,
  'ts': 100409250},
 {'args': {'op_name': 'Mul'},
  'cat': 'Node',
  'dur': 37,
  'name': 'mul_1_kernel_time',
  'ph': 'X',
  'pid': 18,
  'tid': 18,
  'ts': 100409268},
 {'args': {'op_name': 'Mul'},
  'cat': 'Node',
  'dur': 2,
  'name': 'mul_1_fence_after',
  'ph': 'X',
  'pid': 18,
  'tid': 18,
  'ts': 100409314},
 {'args': {},
  'cat': 'Session',
  'dur': 89,
  'name': 'SequentialExecutor::Execute',
  'ph': 'X',
  'pid': 18,
  'tid': 18,
  'ts': 100409239},
 {'args': {},
  'cat': 'Session',
  'dur': 115,
  'name': 'model_run',
  'ph': 'X',
  'pid': 18,
  'tid': 18,
  'ts': 100409224}]


In [22]:
# read in test data protobuf files included with the model
import onnx
from onnx import numpy_helper

In [25]:
import os

In [26]:
tdata_idx = 1

In [27]:
tensor = onnx.TensorProto()
with open(os.path.join(f'/data/mnist/test_data_set_{tdata_idx}/input_0.pb'), 'rb') as f:
    tensor.ParseFromString(f.read())
input_data = numpy_helper.to_array(tensor)

In [28]:
sopts = ort.SessionOptions()
sopts.enable_profiling = True

In [29]:
sess = ort.InferenceSession('/data/mnist/model.onnx', sopts)

In [30]:
in_name = sess.get_inputs()[0].name

In [31]:
out_name = sess.get_outputs()[0].name

In [32]:
r = sess.run([out_name], {in_name : input_data})

In [33]:
perf = sess.end_profiling()

In [34]:
with open(perf, "r") as f:
    sess_time = json.load(f)

In [36]:
len(sess_time)

40

In [37]:
import functools, itertools

In [40]:
different_keys = functools.reduce(set.union, [set(rec.keys()) for rec in sess_time], set())

In [41]:
different_keys

{'args', 'cat', 'dur', 'name', 'ph', 'pid', 'tid', 'ts'}

In [42]:
set(rec['cat'] for rec in sess_time)

{'Node', 'Session'}

In [46]:
all(('op_name' in rec['args']) for rec in sess_time if rec['cat'] == 'Node')

True

In [52]:
[{ "name" : f"{rec['name']}/{rec['args']['op_name']}", "dur" : rec["dur"] }  for rec in sess_time if rec["cat"] == "Node" ]

[{'name': 'Convolution28_fence_before/Conv', 'dur': 6},
 {'name': 'Convolution28_kernel_time/Conv', 'dur': 1702},
 {'name': 'Convolution28_fence_after/Conv', 'dur': 3},
 {'name': 'Plus30_fence_before/Add', 'dur': 3},
 {'name': 'Plus30_kernel_time/Add', 'dur': 24},
 {'name': 'Plus30_fence_after/Add', 'dur': 2},
 {'name': 'ReLU32_fence_before/Relu', 'dur': 3},
 {'name': 'ReLU32_kernel_time/Relu', 'dur': 9},
 {'name': 'ReLU32_fence_after/Relu', 'dur': 2},
 {'name': 'Pooling66_fence_before/MaxPool', 'dur': 2},
 {'name': 'Pooling66_kernel_time/MaxPool', 'dur': 31},
 {'name': 'Pooling66_fence_after/MaxPool', 'dur': 2},
 {'name': 'Convolution110_fence_before/Conv', 'dur': 3},
 {'name': 'Convolution110_kernel_time/Conv', 'dur': 123},
 {'name': 'Convolution110_fence_after/Conv', 'dur': 2},
 {'name': 'Plus112_fence_before/Add', 'dur': 2},
 {'name': 'Plus112_kernel_time/Add', 'dur': 13},
 {'name': 'Plus112_fence_after/Add', 'dur': 2},
 {'name': 'ReLU114_fence_before/Relu', 'dur': 2},
 {'name': 'R